# Key Factor Ranking
Copyright © 2020 AA

In [1]:
%load_ext autoreload
%autoreload 2

import os, warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keyfactordiag.diag import Diag

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 100)
pd.set_option("display.max_colwidth", 100)


### INIT

In [2]:
#init
diag = Diag()
#load Data
diag.load_mesdata()


### Report

In [3]:
#組大表
diag.getbigtable()
#集中性
diag.rankfactor()
#output excel
diag.output_bigtable()
diag.output_factorranktable()

12 features with greater than 0.30 missing values.

79 features with a single unique value.

Removed 84 features.
Total of 84 features identified for removal
['part:BATTERY CABLE:GROUP_NAME', 'part:EDP CABLE:venderid', 'part:AUDIO/B:VERSION', 'part:BOTTOM/B:VERSION', 'part:HDD:GROUP_NAME', 'fatpstation:TP-2:ERROR_FLAG', 'part:AUDIO/B:venderid', 'part:BATTERY CABLE:KEY_PART_NO', 'fatpstation:STRU:LINE_NAME', 'part:KB:GROUP_NAME', 'part:LCD COVER:GROUP_NAME', 'part:TOUCH PAD:GROUP_NAME', 'part:CAMERA:KEY_PART_NO', 'part:BATT:GROUP_NAME', 'smtstation:AOIB:EMP_NO', 'part:SPEAKER:venderid', 'part:LCD PANEL:KEY_PART_NO', 'smtstation:AOIB:ERROR_FLAG', 'part:BOTTOM/B:KEY_PART_NO', 'part:THERMAL:GROUP_NAME', 'part:DC IN CABLE:venderid', 'part:FFC:KEY_PART_NO', 'part:UPPER:venderid', 'fatpstation:TP-2:EMP_NO', 'fatpstation:TP-2:LINE_NAME', 'part:MB:DC', 'fatpstation:STRU:ERROR_FLAG', 'smtstation:AOIA:ERROR_FLAG', 'part:BATTERY CABLE:VERSION', 'part:EDP CABLE:VERSION', 'part:EDP CABLE:GROUP_NAME'

### Test

In [4]:
print('Layer 1的報表:')
diag.factortable['main'].head(15)

Layer 1的報表:


,rank,entropy(mean),Type,Factor A,Factor B,Factor C,Pass,Fail,Others-Pass,Others-Fail
0,0,0.106441,part,MB,GROUP_NAME,PICK2,114,2,2,166
1,1,0.175882,part,MB,GROUP_NAME,PICK5,2,162,114,6
2,2,0.426553,smtstation,ATE,LINE_NAME,3SMTM,12,155,104,13
3,3,0.428248,smtstation,AOIA,LINE_NAME,3SMTM,13,156,103,12
4,4,0.428248,smtstation,ATE,STATION_NAME,ATE1,13,156,103,12
5,5,0.428248,smtstation,AOIB,LINE_NAME,3SMTM,13,156,103,12
6,6,0.439137,smtstation,F/T,LINE_NAME,3PCBM,13,155,103,13
7,7,0.520037,part,AC ADAP,DC,LOC0004E,10,142,106,26
8,8,0.624809,smtstation,ATE,STATION_NAME,P31FIATE03,72,4,44,164
9,9,0.624809,smtstation,AOIB,LINE_NAME,3SMTI,72,4,44,164


In [5]:
print('Layer 2的報表:')
for k in diag.factortable.keys():
    if k=='main':
        continue
    print(k)
    display(diag.factortable[k].head(7))

Layer 2的報表:
0:MB:PICK2


,rank,entropy(mean),Type,Factor A,Factor B,Factor C,Pass,Fail,Others-Pass,Others-Fail
0,0,0.000000,part,POWER CODE,venderid,,0,2,114,0
1,1,0.000000,part,AC ADAP,DC,,0,2,114,0
2,2,0.000000,part,POWER CODE,DC,,0,2,114,0
3,3,0.000000,part,AC ADAP,venderid,,0,2,114,0
4,4,0.059292,part,ANTENNA,KEY_PART_NO,DC330021Y0L,107,0,7,2
5,5,0.059292,part,ANTENNA,KEY_PART_NO,DC330022O0L,7,2,107,0
6,6,0.059292,part,ANTENNA,venderid,ANK00,107,0,7,2


1:MB:PICK5


,rank,entropy(mean),Type,Factor A,Factor B,Factor C,Pass,Fail,Others-Pass,Others-Fail
0,0,0.050509,part,SPEAKER,DC,ZLE0003T,2,12,0,150
1,1,0.053579,part,BATT,KEY_PART_NO,GC02002PV1L,1,0,1,162
2,2,0.053579,part,BATT,VERSION,A01,1,0,1,162
3,3,0.053579,part,BATT,venderid,7EW00,1,0,1,162
4,4,0.053579,part,BATT,DC,7EW0001D,1,0,1,162
5,5,0.056242,part,BATT,VERSION,A02,0,145,2,17
6,6,0.056242,part,BATT,venderid,9ZC00,0,145,2,17


2:ATE:3SMTM


,rank,entropy(mean),Type,Factor A,Factor B,Factor C,Pass,Fail,Others-Pass,Others-Fail
0,0,0.121390,part,MB,GROUP_NAME,PICK2,10,1,2,154
1,1,0.121390,part,MB,GROUP_NAME,PICK5,2,154,10,1
2,2,0.128402,part,BATT,KEY_PART_NO,GC02002JY1L,9,0,3,155
3,3,0.128402,part,BATTERY CABLE,venderid,,9,0,3,155
4,4,0.128402,part,KB,DC,CH200049,3,155,9,0
5,5,0.128402,part,MB,KEY_PART_NO,451AMK31L07,3,155,9,0
6,6,0.128402,part,BATT,VERSION,A05,9,0,3,155


3:AOIA:3SMTM


,rank,entropy(mean),Type,Factor A,Factor B,Factor C,Pass,Fail,Others-Pass,Others-Fail
0,0,0.120840,part,MB,GROUP_NAME,PICK5,2,155,11,1
1,1,0.120840,part,MB,GROUP_NAME,PICK2,11,1,2,155
2,2,0.127046,part,KB,VERSION,A00,3,156,10,0
3,3,0.127046,part,MB,KEY_PART_NO,451A8J31L08,10,0,3,156
4,4,0.127046,part,BATT,VERSION,A05,10,0,3,156
5,5,0.127046,part,KB,DC,CH200049,3,156,10,0
6,6,0.127046,part,KB,venderid,DFH00,10,0,3,156


4:ATE:ATE1


,rank,entropy(mean),Type,Factor A,Factor B,Factor C,Pass,Fail,Others-Pass,Others-Fail
0,0,0.120840,part,MB,GROUP_NAME,PICK5,2,155,11,1
1,1,0.120840,part,MB,GROUP_NAME,PICK2,11,1,2,155
2,2,0.127046,part,KB,VERSION,A00,3,156,10,0
3,3,0.127046,part,MB,KEY_PART_NO,451A8J31L08,10,0,3,156
4,4,0.127046,part,BATT,VERSION,A05,10,0,3,156
5,5,0.127046,part,KB,DC,CH200049,3,156,10,0
6,6,0.127046,part,KB,venderid,DFH00,10,0,3,156


***